In [ ]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 1.6 MB/s 
     |████████████████████████████████| 5.5 MB 34.8 MB/s 
     |████████████████████████████████| 1.3 MB 54.9 MB/s 
     |████████████████████████████████| 163 kB 46.0 MB/s 
     |████████████████████████████████| 7.6 MB 36.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=268ca3509821f2911526e21f13e82b95bd033c69ffff84ddf3322b0fd565da85
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [ ]:
!cp -r drive/MyDrive/Data .

In [ ]:
from sentence_transformers import SentenceTransformer
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer  
from tqdm import tqdm_notebook, tnrange
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import seaborn as sn
import matplotlib.pyplot as plt

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


In [ ]:
df_data = pd.read_csv("Data/arguments-training.tsv", sep='\t')

In [ ]:
df_data

,Argument ID,Conclusion,Stance,Premise
0,A01001,Entrapment should be legalized,in favor of,if entrapment can serve to more easily capture...
1,A01002,We should ban human cloning,in favor of,we should ban human cloning as it will only ca...
2,A01003,We should abandon marriage,against,marriage is the ultimate commitment to someone...
3,A01004,We should ban naturopathy,against,it provides a useful income for some people
4,A01005,We should ban fast food,in favor of,fast food should be banned because it is reall...
...,...,...,...,...
5215,D27096,Nepotism exists in Bollywood,against,Star kids also have an upbringing which is sur...
5216,D27097,Nepotism exists in Bollywood,in favor of,Movie stars of Bollywood often launch their ch...
5217,D27098,India is safe for women,in favor of,Evil historic practices on women in the pre an...
5218,D27099,India is safe for women,in favor of,Women of our country have been and are achievi...


In [ ]:
df_data['text'] = df_data['Premise']+' '+df_data['Stance']+' '+df_data['Conclusion']

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm", disable=['parser', 'tagger', 'ner'])
stops = nlp.Defaults.stop_words


def normalize(text, lowercase, remove_stopwords):
    if lowercase:
        text = text.lower()
    text = nlp(text)
    lemmatized = list()
    for word in text:
        lemma = word.lemma_.strip()
        if lemma:
            if not remove_stopwords or (remove_stopwords and lemma not in stops):
                lemmatized.append(lemma)
    return " ".join(lemmatized)


df_data['text'] = df_data['text'].apply(normalize, lowercase=True, remove_stopwords=True)


/usr/local/lib/python3.7/dist-packages/spacy/language.py:1899: UserWarning: [W123] Argument disable with value ['parser', 'tagger', 'ner'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  config_value=config["nlp"][key],
/usr/local/lib/python3.7/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [ ]:
data = df_data['text'].to_numpy()
data

array(['entrapment serve easily capture wanted criminals , legal ? favor entrapment legalized',
       'ban human cloning cause huge issues bunch humans running acting . favor ban human cloning',
       'marriage ultimate commitment , people want allowrd abandon marriage',
       ...,
       'evil historic practices women pre post medieval period oblivion . practices like child marriages , polygamy , ban remarriages , honor killings , sati , jauhar purdah obsolete today world . time , practices redundant , increasing safety measures . favor india safe women',
       'women country achieving epitomes success glory day . prominent personalities like savitribai phule , mother teresa , sarla thakral , kalpana chawla , sister nivedita , sarojini naidu , conquering respective fields centuries . recent times , prativa patil , meira kumar , priyanka n. , mitali madhumita , tanushree pareek women achievers queens . accomplishments happening , safety guaranteed big extent . favor india safe wome

In [ ]:
df_label = pd.read_csv("Data/labels-training.tsv", sep='\t')

In [ ]:
df_label

,Argument ID,Self-direction: thought,Self-direction: action,Stimulation,Hedonism,Achievement,Power: dominance,Power: resources,Face,Security: personal,...,Tradition,Conformity: rules,Conformity: interpersonal,Humility,Benevolence: caring,Benevolence: dependability,Universalism: concern,Universalism: nature,Universalism: tolerance,Universalism: objectivity
0,A01001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,A01002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,A01003,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,A01004,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,A01005,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5215,D27096,1,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5216,D27097,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
5217,D27098,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5218,D27099,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
df_label[df_label == 1].sum()

Argument ID                        0
Self-direction: thought        913.0
Self-direction: action        1332.0
Stimulation                    312.0
Hedonism                       202.0
Achievement                   1400.0
Power: dominance               461.0
Power: resources               566.0
Face                           374.0
Security: personal            1961.0
Security: societal            1627.0
Tradition                      598.0
Conformity: rules             1222.0
Conformity: interpersonal      217.0
Humility                       438.0
Benevolence: caring           1500.0
Benevolence: dependability     766.0
Universalism: concern         1992.0
Universalism: nature           358.0
Universalism: tolerance        709.0
Universalism: objectivity      937.0
dtype: object

In [ ]:
label = df_label.loc[:, df_label.columns != 'Argument ID'].to_numpy()
label

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [ ]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(data, label, test_size=0.2)

In [ ]:
print(test_x[:10])
print(test_y[:10])

['minors mature decisions favor ban cosmetic surgery minors'
 'strengthen regulation adjustment high income , protect legitimate income according law , reasonably adjust excessive income , encourage high - income people enterprises society . favor crack unreasonably high incomes .'
 'time child foster care gets separated siblings . favor foster care brings harm good'
 'vocational education option schools discourages students pursuing higher education degrees form universities subsidize vocational education'
 'factory farming cuts health safety corners opposed smaller family farms , leads potentially unsafe unhealthy products . favor ban factory farming'
 'racial profiling guarantee crime prevented stopped . favor end racial profiling'
 'autonomous cars help people disabled able transportation . stop development autonomous cars'
 'sex selection create places boys outnumber girls lot places value boys girls , create problems older legalize sex selection'
 'legalizing prostitution sex wor

In [ ]:
labse_model = SentenceTransformer('sentence-transformers/LaBSE')
train_x = labse_model.encode(train_x)
test_x = labse_model.encode(test_x)
# print(labse_embeddings)

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/114 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.62M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.22M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/461 [00:00<?, ?B/s]

In [ ]:
print("Train shape : ",train_x.shape)
print("Test shape : ",test_x.shape)
print("Train shape : ",train_y.shape)
print("Test shape : ",test_y.shape)

Train shape :  (4176, 768)
Test shape :  (1044, 768)
Train shape :  (4176, 20)
Test shape :  (1044, 20)


In [ ]:
class Baseline(nn.Module):
    
    def __init__(self, n_classes, emb_size):
        super(Baseline, self).__init__()
        self.dropout = nn.Dropout(0.1)
        self.fc1 = nn.Linear(emb_size, 500)
        self.fc2 = nn.Linear(500, 300)
        self.fc3 = nn.Linear(300, 100)
        self.fc4 = nn.Linear(100, 50)
        self.fc5 = nn.Linear(50, n_classes)
        self.sigm = nn.Sigmoid()


    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x) 
        x = self.fc2(x) 
        x = F.relu(x)
        x = self.fc3(x) 
        x = F.relu(x)
        x = self.fc4(x) 
        x = F.relu(x)
        x = self.fc5(x) 
        return self.sigm(x)

In [ ]:
n_epochs = 10
model = Baseline(20, train_x.shape[1])
criterion = nn.BCELoss()
batch_size = 8
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
model.to(device)
model.train()


train_x = torch.tensor(train_x)
train_y = torch.tensor(train_y)
test_x = torch.tensor(test_x)
test_y = torch.tensor(test_y)

train = torch.utils.data.TensorDataset(train_x, train_y)
test = torch.utils.data.TensorDataset(test_x, test_y)

train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)

epoch = 0

while True:
    batch_losses = []
    for x, y in train_loader:
        x.to(device)
        y.to(device)
 
        optimizer.zero_grad()
 
        out = model(x)
        loss = criterion(out, y.type(torch.float))
 
        batch_loss_value = loss.item()
        loss.backward()
        optimizer.step()
 
        batch_losses.append(batch_loss_value)
 
    loss_value = np.mean(batch_losses)
    print("epoch:{:2d} train: loss:{:.3f}".format(epoch, loss_value))
    # if epoch % save_freq == 0:
    #     checkpoint_save(model, save_path, epoch)
    epoch += 1
    if n_epochs < epoch:
        break

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is 

epoch: 0 train: loss:0.429
epoch: 1 train: loss:0.388
epoch: 2 train: loss:0.367
epoch: 3 train: loss:0.353
epoch: 4 train: loss:0.340
epoch: 5 train: loss:0.329
epoch: 6 train: loss:0.316
epoch: 7 train: loss:0.306
epoch: 8 train: loss:0.294
epoch: 9 train: loss:0.282
epoch:10 train: loss:0.270


In [ ]:
from sklearn.metrics import f1_score, recall_score, precision_score

model.eval()
preds = []
targets = []

with torch.no_grad():
    for x, y in test_loader:
        x.to(device)
        out = model(x)
        out = torch.where(out>=0.5, 1, 0)
        # print(out)
        # print(y)
        for i in out.cpu().numpy():
            preds.append(i)
        for i in y.cpu().numpy():
            targets.append(i)
        # print(out)
preds = np.asarray(preds)
targets = np.asarray(targets)
# print(preds.T[i])
# print(preds[:][0].shape)
# print(targets[:][0].shape)
for i in range(20):
    # recall = recall_score(y_true=targets.T[i], y_pred=preds.T[i])
    # precision = precision_score(y_true=targets.T[i], y_pred=preds.T[i])
    print(f'{i}: ')
    print(f1_score(y_true=targets.T[i], y_pred=preds.T[i]))


0: 
0.6315789473684211
1: 
0.5753424657534246
2: 
0.09677419354838708
3: 
0.25
4: 
0.5591836734693877
5: 
0.10101010101010101
6: 
0.5196078431372548
7: 
0.06349206349206349
8: 
0.6850393700787402
9: 
0.624561403508772
10: 
0.5123152709359606
11: 
0.41578947368421054
12: 
0.14285714285714288
13: 
0.0
14: 
0.5066666666666666
15: 
0.014084507042253521
16: 
0.6271676300578035
17: 
0.682170542635659
18: 
0.3560606060606061
19: 
0.3359375


In [ ]:
d = {}
j = 0
for i in df_label.columns[1:]:
    d[j] = i
    j+=1

In [ ]:
d

{0: 'Self-direction: thought',
 1: 'Self-direction: action',
 2: 'Stimulation',
 3: 'Hedonism',
 4: 'Achievement',
 5: 'Power: dominance',
 6: 'Power: resources',
 7: 'Face',
 8: 'Security: personal',
 9: 'Security: societal',
 10: 'Tradition',
 11: 'Conformity: rules',
 12: 'Conformity: interpersonal',
 13: 'Humility',
 14: 'Benevolence: caring',
 15: 'Benevolence: dependability',
 16: 'Universalism: concern',
 17: 'Universalism: nature',
 18: 'Universalism: tolerance',
 19: 'Universalism: objectivity'}

In [ ]:
sample = ['everybody different & people like express clothes , wearing uniform allow person individuality favor abandon use school uniform'
 ,'making child work actor deprives childhood experiences interactions . favor ban use child actors'
 ,'- strikes law good keeping criminals streets abolish - strikes laws'
 ,'school uniforms positive effect academic achievement . favor abandon use school uniform'
 ,'abolish - strikes laws stick law , stop reoffending harsh favor abolish - strikes laws'
 ,'prey poor charge astronomic interest . favor payday loans banned'
 ,'olympics positive athletic contest builds pride country promotes cooperation participating countries . goodwill ambassador world remain . abolish olympic games'
 ,'blockade gaza strip reasonable response crimes committed hamas members . blockade gaza strip ended'
 ,'pride parades grown point increasingly difficult police scale events need looked ensure safety participants . favor cancel pride parades'
 ,'adopt atheism believe . adopt atheism']

target = [[1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
target2 = []
for i in target:
    tmp = ''
    for j in range(len(i)):
        if i[j]==1:
            tmp=tmp+d[j]+'/'
    target2.append(tmp)
# for x in sample:
sample = labse_model.encode(sample)
x = torch.Tensor(sample)
x.to(device)
out = model(x)
pred = np.array(out > 0.5, dtype=float)
# print(pred)
pred2 = []
for i in pred:
    tmp = ''
    for j in range(len(i)):
        if i[j]==1:
            tmp=tmp+d[j]+'/'
    pred2.append(tmp)
print(target2)
print(pred2)

['Self-direction: thought/Self-direction: action/Face/Universalism: tolerance/', 'Hedonism/Benevolence: caring/', 'Security: societal/Conformity: rules/Universalism: concern/', 'Achievement/Benevolence: caring/Universalism: objectivity/', 'Power: dominance/Security: societal/Tradition/Conformity: rules/Universalism: concern/', 'Power: resources/Security: personal/Conformity: rules/Benevolence: dependability/Universalism: concern/', 'Security: personal/Security: societal/Tradition/Universalism: concern/', 'Power: dominance/Security: societal/Conformity: rules/Universalism: concern/', 'Security: personal/Security: societal/', 'Self-direction: thought/Self-direction: action/Hedonism/Tradition/']
['Self-direction: thought/Self-direction: action/Universalism: tolerance/', 'Security: personal/Benevolence: caring/', 'Security: societal/Conformity: rules/Universalism: concern/', 'Achievement/Universalism: tolerance/Universalism: objectivity/', 'Security: societal/Universalism: concern/', 'Powe